## Introduksjon av feil/errors i dataframes og retting av feil/errors

Værdataen vi har samlet er komplett, og inneholder ikke uteliggere, feilverdier eller lignende. Luftkvalitetdataen har en god del "null" verdier, men ellers er den ryddig også. Vi kan derfor manuelt introdusere diverse feil i datasettene og rette de opp igjen ved bruk av funksjoner fra numpy for eksempel. 

Vi lagrer de nye datasettene med feil i data folderen, og bruker nye funksjoner til å rette opp i det igjen. Merk at for Værdataen, kommer vi til å bruke den originale dataen (df2_data) til visualisering og prediktiv analyse, mens for luftkvalitetdataen vil vi bruke den rettet opp filen (df1_fikset), ettersom den originale inneholder nullverdier. 

Vi begynner med å introdusere forskjellige typer feilverdier til værdataen (df2_data): 

In [1]:
import pandas as pd
import sys

df2_errors = pd.read_json("../data/df2_data.json") #leser json filen "df2_data"

sys.path.append('../src')
from funksjoner import introduser_NaN, introduser_uteliggere #Importerer funksjoner fra "funksjoner.py i src"

nan = introduser_NaN(df2_errors) #Introduserer numpy NaN verdier
uteligger = introduser_uteliggere(nan) #Introduserer uteliggere

uteligger.to_json('../data/df2_errors.json', indent=4) #Lagrer den nye dataframe med errors til ny fil "df2_errors.json"

Nå har vi værdata med manglende verdier og uteliggere, som vi kan så rette opp i ved bruk av diverse teknikker: 

Vi kan bruke ffill, bfill, median, mean og interpolate til å fikse på manglende verdier, og setter øvre og nedre grenser for å håndtere uteliggere. 

Under har vi brukt "mean" metoden for manglende data og filtrering med øvre og nedre grenser

In [ ]:
import pandas as pd
import sys

df2_fikset=pd.read_json("../data/df2_errors.json") #Leser json filen "df2_errors"

sys.path.append('../src')
from funksjoner import håndter_mangler, håndter_uteliggere



mangler=håndter_mangler(df2_fikset,"mean(relative_humidity P1D)","mean") #Bruker gjennomsnittet til å fylle inn manglende verdier

#Definerer øvre og nedre grense for å kvitte oss med verdier utenfor de grensene
øvre_grense=60
nedre_grense=-25

uteliggere=håndter_uteliggere(mangler,"mean(air_temperature P1D)",øvre_grense,nedre_grense) #Fjerner uteliggere fra datsett uten manglende verdier

uteliggere.to_json('../data/df2_fikset.json',indent=4) #Lagrer fikset data til "df2_fikset.json"



Videre kan vi se på luftkvalitetdataen: 

Her har vi data som mangler ("null" verdier), som kan ses i "df1_data". Problemet er at nullverdiene ikke er spredt ut i dataen men i blokker, der mange dager på rad har manglende verdier. 



In [10]:
df1_fikset=pd.read_json('../data/df1_data.json')
df1_fikset['date']=pd.to_datetime(df1_fikset['date'],unit='ms')

for column in df1_fikset:
    if column != "date":
        df1_fikset=håndter_mangler(df1_fikset,column,'interpolate')

df1_fikset.to_json('../data/df1_fikset.json',indent=4)
